In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, LSTM, GRU, Dense

In [21]:
num_words= 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [3]:
maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [4]:
embedding =100
model = Sequential([
    Embedding(input_dim=num_words, output_dim=embedding, input_length=maxlen),
    Bidirectional(LSTM(5)),
    Dense(1, activation='sigmoid')
])
model.build()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=num_words,
                    epochs=1)
model.summary()


c:\Users\HP\Desktop\VSCode\Bidirection_lstm\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 688ms/step - accuracy: 0.5341 - loss: 0.6916


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [6]:
train_loss, train_acc = model.evaluate(x_test, y_test, verbose=2)

782/782 - 3s - 3ms/step - accuracy: 0.6084 - loss: 0.6849


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [8]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [9]:
log_dir

'logs/fit20240723-171905'

In [10]:
model_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
import mlflow

In [13]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[model_callbacks, earLy_stop]
)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.7445 - loss: 0.5278 - val_accuracy: 0.8082 - val_loss: 0.4327
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.8815 - loss: 0.2992 - val_accuracy: 0.8097 - val_loss: 0.4209
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9230 - loss: 0.2119 - val_accuracy: 0.8042 - val_loss: 0.5125
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.9481 - loss: 0.1466 - val_accuracy: 0.7995 - val_loss: 0.5369
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9659 - loss: 0.1032 - val_accuracy: 0.7962 - val_loss: 0.5978
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.9794 - loss: 0.0710 - val_accuracy: 0.7910 - val_loss: 0.8143
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.9842 - loss: 0.0528 - val_accuracy: 0.7912 - val_loss: 0.7843
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.9922 - loss: 0

In [11]:
earLy_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [17]:
from mlflow.models.signature import infer_signature
model_sign = infer_signature(x_test, model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [18]:
exp_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name= "IMDB_bidirection_lstm_time:" + exp_timestamp
#exp_name = "IMDB_bidirection_lstm"
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = "IMDB_bidirection_lstm_run:" + run_timestamp

In [22]:
def tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", num_words )
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", 100)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.tensorflow.log_model(model, "IMDB", signature=model_sign)
    mlflow.end_run()

In [23]:
tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign)

2024/07/23 17:35:05 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_bidirection_lstm_time:20240723-173159' does not exist. Creating a new experiment.


In [14]:
model.save("models/model.h5")